# intermediate_hellaswag
This notebook takes our hellaswag dataset and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [3]:
import os
os.chdir('..')
# os.getcwd( )

In [4]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd
from datasets import load_from_disk

from transformers import RobertaTokenizer, RobertaForMultipleChoice
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [5]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 4
params.output_dir = "model_saves/intermediate_HellaSwag_01"
params.dataset_path = "data/inter_HellaSwag/itesd_hellaswag_balanced.hf"

# Ensure we're on an ARM environment if necessary.
platform_check()

We're Armed: macOS-13.1-arm64-i386-64bit


## Load Data

### hellaswag

In [4]:
hellaswag_datasets = load_from_disk(params.dataset_path)

In [5]:
hellaswag_datasets

DatasetDict({
    train: Dataset({
        features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label', 'ending0', 'ending1', 'ending2', 'ending3'],
        num_rows: 34600
    })
    validation: Dataset({
        features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label', 'ending0', 'ending1', 'ending2', 'ending3'],
        num_rows: 10042
    })
    test: Dataset({
        features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label', 'ending0', 'ending1', 'ending2', 'ending3'],
        num_rows: 4994
    })
})

In [6]:
def show_one(example):
    print(f"Context: {example['ctx_a']}")
    print(f"  A - {example['ctx_b']} {example['ending0']}")
    print(f"  B - {example['ctx_b']} {example['ending1']}")
    print(f"  C - {example['ctx_b']} {example['ending2']}")
    print(f"  D - {example['ctx_b']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

show_one(hellaswag_datasets["train"][50])

Context: A woman in a green shirt stands in a kitchen.
  A - she stands in front of several ingredients.
  B - she starts mixing other ingredients in a box.
  C - she begins by taking some green eye shadow.
  D - she shows off a saucepan full of sauerkraut.

Ground truth: option A


## Preprocess

In [7]:
params.tokenizer = RobertaTokenizer.from_pretrained("roberta-base", use_fast=True)

In [ ]:
# encoding_dict
encoded_datasets = hellaswag_datasets.map(mc_preprocessing, batched=True)
encoded_datasets

### Double-Check input_id lengths
We're performing this check to ensure that 256 max token length is sufficient for this task.

In [9]:
train_ids = encoded_datasets["train"]['input_ids']

lengths = []
for i in train_ids:
    for j in i:
        lengths.append(len(j))

print(len(lengths))

138400


In [10]:
max(lengths)

159

### View input Structure

The inputs are four copies of cxt_a and ctx_b each strung together with one ending option. They start with the \<s> BOS token, which may act as the CLS token instead, and are separated with the \</s> token--end of sequence or separator token.

https://huggingface.co/docs/transformers/model_doc/roberta
https://stackoverflow.com/questions/61465223/roberta-tokenization-of-multiple-sequences

In [11]:
show_one(hellaswag_datasets["train"][0])

Context: [header] How to tell when lotus root has gone bad [title] Look for mold or mildew. [step] Examine the root for signs of mold or mildew. Any dark green, black, gray or white growth on the outside of the root is an indication that it has spoiled.
  A -  If your root has mold or mildew on it, you should discard it. [title] Observe to see if the root is soft.
  B -  [substeps] Rose dimes, or similarly rancid looking projections, should not be given to toddlers, as they may have a bacterial or fungal infection. If no mold develops, the roots of the plant are likely to rot.
  C -  If this occurs, it has gone bad. Mildew, for instance, is typically mold-like, and will produce dark, black growths on the plant and/or buds.
  D -  [substeps] Mildew may also be a sign of rot. Look for scorch marks or discoloration on new roots, that may indicate root rot or mildew.

Ground truth: option A


In [12]:
print(params.tokenizer.decode(encoded_datasets['train']["input_ids"][0][0]))
print(params.tokenizer.decode(encoded_datasets['train']["input_ids"][0][1]))
print(params.tokenizer.decode(encoded_datasets['train']["input_ids"][0][2]))
print(params.tokenizer.decode(encoded_datasets['train']["input_ids"][0][3]))

<s>[header] How to tell when lotus root has gone bad [title] Look for mold or mildew. [step] Examine the root for signs of mold or mildew. Any dark green, black, gray or white growth on the outside of the root is an indication that it has spoiled.</s></s> If your root has mold or mildew on it, you should discard it. [title] Observe to see if the root is soft.</s>
<s>[header] How to tell when lotus root has gone bad [title] Look for mold or mildew. [step] Examine the root for signs of mold or mildew. Any dark green, black, gray or white growth on the outside of the root is an indication that it has spoiled.</s></s> [substeps] Rose dimes, or similarly rancid looking projections, should not be given to toddlers, as they may have a bacterial or fungal infection. If no mold develops, the roots of the plant are likely to rot.</s>
<s>[header] How to tell when lotus root has gone bad [title] Look for mold or mildew. [step] Examine the root for signs of mold or mildew. Any dark green, black, gr

## Prep Dataloaders


In [13]:
train_features = construct_input(encoded_datasets['train'])
validate_features = construct_input(encoded_datasets['validation'])

In [14]:
# dataloaders w collation
# Prepare DataLoader
train_dataloader = DataLoader(
            train_features,
            sampler = RandomSampler(train_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

validation_dataloader = DataLoader(
            validate_features,
            sampler = RandomSampler(validate_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

In [15]:
# view an example from the dataloader
next(iter(train_dataloader))

[tensor([[[    0, 10975, 24419,  ...,     1,     1,     1],
          [    0, 10975, 24419,  ...,     1,     1,     1],
          [    0, 10975, 24419,  ...,     1,     1,     1],
          [    0, 10975, 24419,  ...,     1,     1,     1]],
 
         [[    0,   250,   340,  ...,     1,     1,     1],
          [    0,   250,   340,  ...,     1,     1,     1],
          [    0,   250,   340,  ...,     1,     1,     1],
          [    0,   250,   340,  ...,     1,     1,     1]],
 
         [[    0, 10975, 24419,  ...,     1,     1,     1],
          [    0, 10975, 24419,  ...,     1,     1,     1],
          [    0, 10975, 24419,  ...,     1,     1,     1],
          [    0, 10975, 24419,  ...,     1,     1,     1]],
 
         ...,
 
         [[    0,   243,   924,  ...,     1,     1,     1],
          [    0,   243,   924,  ...,     1,     1,     1],
          [    0,   243,   924,  ...,     1,     1,     1],
          [    0,   243,   924,  ...,     1,     1,     1]],
 
         [[ 

## Train

* Note: if continuing from checkpoint, continue to next section

Download transformers.RobertaForSequenceClassification, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [6]:
# Load the RobertaForSequenceClassification model
model = RobertaForMultipleChoice.from_pretrained('roberta-base',
                                                  num_labels = params.num_labels,
                                                  output_attentions = False,
                                                  output_hidden_states = False,
                                                    )

from torchinfo import summary
summary(model, input_size=(1, 4, 256), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForMultipleChoice                                     [1, 4]                    --
├─RobertaModel: 1-1                                          [4, 768]                  --
│    └─RobertaEmbeddings: 2-1                                [4, 256, 768]             --
│    │    └─Embedding: 3-1                                   [4, 256, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [4, 256, 768]             768
│    │    └─Embedding: 3-3                                   [4, 256, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [4, 256, 768]             1,536
│    │    └─Dropout: 3-5                                     [4, 256, 768]             --
│    └─RobertaEncoder: 2-2                                   [4, 256, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [17]:
model.to(params.device)
# print(f"Trained Dataset: {dataset_path}")
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), 
                             lr=params.learning_rate,
                             weight_decay=params.weight_decay) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

output_parameters()

Device: mps

          Training Dataset: data/inter_HellaSwag/itesd_hellaswag_balanced.hf
          Number of Labels: 4
          Batch Size: 16
          Learning Rate: 1e-05
          Weight Decay: 0
          Epochs: 10
          Output Directory: model_saves/intermediate_HellaSwag_01
          Save Frequency: 1
          Checkpoint Frequency: 1
          Max Length: 256
          


In [ ]:
trainer.fit()

## Continue Training from Checkpoint

In [16]:
# Load the RobertaForSequenceClassification model
model = RobertaForMultipleChoice.from_pretrained('roberta-base',
                                                  num_labels = params.num_labels,
                                                  output_attentions = False,
                                                  output_hidden_states = False,
                                                    )

model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), 
                             lr=params.learning_rate,
                             weight_decay=params.weight_decay) #roberta

checkpoint_load = "model_saves/intermediate_HellaSwag_01/E06_A0.61_F0.61/checkpoint.pt"
trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq, 
                  checkpoint_load=checkpoint_load)

Device: mps


In [17]:
trainer.fit()

	 Validation 627: 100%|██████████| 628/628 [10:01<00:00,  1.04batch/s]

 	 - Train loss: 0.165883
	 - Validation Loss: 1.644658
	 - Validation Accuracy: 0.618451
	 - Validation F1: 0.618451
	 - Validation Recall: 0.618451
	 - Validation Precision: 0.618451 

	 * Model @ epoch 7 saved to model_saves/intermediate_HellaSwag_01/E07_A0.62_F0.62
	 * Model checkpoint saved to model_saves/intermediate_HellaSwag_01/E07_A0.62_F0.62/checkpoint.pt

	 Validation 627: 100%|██████████| 628/628 [10:01<00:00,  1.04batch/s]

 	 - Train loss: 0.139275
	 - Validation Loss: 1.798362
	 - Validation Accuracy: 0.606768
	 - Validation F1: 0.606768
	 - Validation Recall: 0.606768
	 - Validation Precision: 0.606768 

	 * Model @ epoch 8 saved to model_saves/intermediate_HellaSwag_01/E08_A0.61_F0.61
	 * Model checkpoint saved to model_saves/intermediate_HellaSwag_01/E08_A0.61_F0.61/checkpoint.pt

	 Validation 627: 100%|██████████| 628/628 [09:58<00:00,  1.05batch/s]

 	 - Train loss: 0.112031
	 - Validation Loss:

In [1]:
token_ids = [[23,42,42,23], [23,43,23,43]]
attention_masks = [[1,1,1,1], [1,1,1,1]]

test = {z[0]: list(z[1:]) for z in zip(token_ids, attention_masks)}

test

TypeError: unhashable type: 'list'